In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

import sys
import xgboost as xgb
#!{sys.executable} -m pip install xgboost

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')

In [3]:
df = df.drop(columns = ['deaths'])

In [4]:
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')

In [5]:
df['fips'] = df['fips'].fillna(0)
df['fips'] = df['fips'].astype(int)
df['fips'] = df['fips'].astype(str)
df['fips'] = df['fips'].str.zfill(5)

In [6]:
#Assign New York to a unique FIPS
df.loc[df['county'] == 'New York City','fips'] = '36999'

#Assign unknown counties to a generic FIPS code for state
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Alabama'),'fips'] = '01000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Alaska'),'fips'] = '02000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Arizona'),'fips'] = '04000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Arkansas'),'fips'] = '05000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'California'),'fips'] = '06000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Colorado'),'fips'] = '08000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Connecticut'),'fips'] = '09000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Delaware'),'fips'] = '10000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Florida'),'fips'] = '12000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Georgia'),'fips'] = '13000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Hawaii'),'fips'] = '15000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Idaho'),'fips'] = '16000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Illinois'),'fips'] = '17000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Indiana'),'fips'] = '18000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Iowa'),'fips'] = '19000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Kansas'),'fips'] = '20000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Kentucky'),'fips'] = '21000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Louisiana'),'fips'] = '22000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Maine'),'fips'] = '23000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Maryland'),'fips'] = '24000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Massachusetts'),'fips'] = '25000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Michigan'),'fips'] = '26000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Minnesota'),'fips'] = '27000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Mississippi'),'fips'] = '28000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Missouri'),'fips'] = '29000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Montana'),'fips'] = '30000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Nebraska'),'fips'] = '31000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Nevada'),'fips'] = '32000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'New Hampshire'),'fips'] = '33000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'New Jersey'),'fips'] = '34000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'New Mexico'),'fips'] = '35000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'New York'),'fips'] = '36000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'North Carolina'),'fips'] = '37000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'North Dakota'),'fips'] = '38000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Ohio'),'fips'] = '39000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Oklahoma'),'fips'] = '40000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Oregon'),'fips'] = '41000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Pennsylvania'),'fips'] = '42000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Rhode Island'),'fips'] = '44000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'South Carolina'),'fips'] = '45000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'South Dakota'),'fips'] = '46000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Tennessee'),'fips'] = '47000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Texas'),'fips'] = '48000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Utah'),'fips'] = '49000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Vermont'),'fips'] = '50000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Virginia'),'fips'] = '51000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Washington'),'fips'] = '53000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'West Virginia'),'fips'] = '54000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Wisconsin'),'fips'] = '55000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Wyoming'),'fips'] = '56000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'American Samoa'),'fips'] = '60000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Guam'),'fips'] = '66000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Northern Mariana Islands'),'fips'] = '69000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Puerto Rico'),'fips'] = '72000'
df.loc[(df['county'] == 'Unknown') & (df['state'] == 'Virgin Islands'),'fips'] = '78000'

#Assign cities in Missouri based on the county the mostly reside in.
df.loc[(df['county'] == 'Joplin') & (df['state'] == 'Missouri'),'fips'] = '29998'
df.loc[(df['county'] == 'Kansas City') & (df['state'] == 'Missouri'),'fips'] = '29999'

In [7]:
df_left = df.copy()
df_left['date'] = df_left['date'] + pd.Timedelta(days = 1)

In [8]:
df_diff = df.merge(df_left[['date', 'fips', 'cases']], on = ['date', 'fips'], how = 'left')

In [9]:
df_diff['cases_y'] = df_diff['cases_y'].fillna(0)
df_diff['cases'] = df_diff['cases_x'] - df_diff['cases_y']
df_diff = df_diff.drop(columns = ['cases_x', 'cases_y'])

In [10]:
#Purpose is to test reasonability of model based on past surges.
#We only expected to see a Delta Surge in June for specific regions, like Springfield Missouri.
#Delta surge will be much more widespread in July
#If our model predicts this behavior, then the results are reasonable.
df_diff = df_diff.loc[df_diff['date'] <= pd.to_datetime('2021-06-01', format = '%Y-%m-%d')]

In [11]:
# data field documentation: 
# https://svi.cdc.gov/Documents/Data/2016_SVI_Data/SVI2016Documentation.pdf
svi = pd.read_csv('https://svi.cdc.gov/Documents/Data/2018_SVI_Data/CSV/SVI2018_US_COUNTY.csv')
#Convert to lowercase for better handling when casting CSR matrix
svi.columns = svi.columns.str.lower()
#Pad leading zeros for FIPS codes
svi['fips'] = svi['fips'].astype(int)
svi['fips'] = svi['fips'].astype(str)
svi['fips'] = svi['fips'].str.zfill(5)
#Only use population for now, plan to add more fields from SVI data later
svi = svi[['fips', 'e_totpop']]

In [12]:
#fips codes for New York City Boroughs
svi_nyc = svi.loc[svi['fips'].isin(['36085', '36061', '36047', '36081', '36005'])]
svi_nyc = svi_nyc.assign(fips = '36999')
svi_nyc = svi_nyc.groupby(['fips'], as_index = False).sum()

In [13]:
#append original data with aggregated NYC data
svi = svi.loc[~svi['fips'].isin(['36085', '36061', '36047', '36081', '36005'])]
svi = svi.append(svi_nyc)

In [14]:
df_target_denom = pd.DataFrame()
for x in df_diff['date'].unique():
    df_diff_temp = df_diff.loc[df_diff['date'] <= x]
    
    sorted_val = df_diff_temp.loc[df_diff_temp['cases'] > 0].sort_values(by='date')
    recent_result_cases = sorted_val.drop_duplicates('fips', keep='last')

    df_diff_temp = df_diff_temp.loc[df_diff_temp['date'] == x]

    df_cases_target = df_diff_temp.merge(recent_result_cases[['date', 'fips', 'cases']], on = ['fips'], how = 'left')
    df_cases_target['days_since_gt0'] = df_cases_target['date_x'] - df_cases_target['date_y']
    df_cases_target.rename(columns={'date_x':'date', 'cases_y':'recent_cases_gt0'}, inplace=True)
    df_cases_target = df_cases_target.drop(columns = ['date_y', 'cases_x'])
    df_target_denom = df_target_denom.append(df_cases_target)
df_target_denom = df_target_denom.dropna()

In [15]:
df_cases_lag = \
    df_diff.merge(df_target_denom[['date', 'fips', 'recent_cases_gt0', 'days_since_gt0']], on = ['date', 'fips'], how = 'inner')

In [16]:
#add day_of_week field
df_cases_lag['day_of_week']=df_cases_lag['date'].dt.dayofweek
#convert day_of_week to Int
df_cases_lag['day_of_week'] =df_cases_lag['day_of_week'].astype(int)
#rename day_of_week field 
days = {0:'0_Mon',1:'1_Tues',2:'2_Weds',3:'3_Thurs',4:'4_Fri',5:'5_Sat',6:'6_Sun'}
df_cases_lag['day_of_week'] = df_cases_lag['day_of_week'].apply(lambda x: days[x])
df_cases_lag['month']=pd.DatetimeIndex(df_cases_lag['date']).month

In [17]:
df_left = df_cases_lag.copy()
for i in range(1, 31):
    df_left['date'] = df_left['date'] - pd.Timedelta(days = -1)
    df_cases_lag = df_cases_lag.merge(df_left[['date', 'fips', 'cases']], on = ['date', 'fips'], how = 'inner')
    df_cases_lag.rename(columns={'cases_y':'cases' + str(i).zfill(2)}, inplace=True)
    df_cases_lag.rename(columns={'cases_x':'cases'}, inplace=True)
df_cases_lag.rename(columns={'cases':'cases00'}, inplace=True)

In [18]:
for i in range(30):
    df_cases_lag['cases'  + str(i).zfill(2) + '_norm'] = df_cases_lag['cases' + str(i).zfill(2)] / df_cases_lag['recent_cases_gt0']
for i in range(29):
    df_cases_lag['cases'  + str(i).zfill(2) + '_deriv1_norm'] = \
        df_cases_lag['cases'  + str(i).zfill(2) + '_norm'] \
            - df_cases_lag['cases'  + str(i + 1).zfill(2) + '_norm']
for i in range(28):
    df_cases_lag['cases'  + str(i).zfill(2) + '_deriv2_norm'] = \
        df_cases_lag['cases'  + str(i).zfill(2) + '_deriv1_norm'] \
            - df_cases_lag['cases'  + str(i + 1).zfill(2) + '_deriv1_norm']

C:\Users\gordo\AppData\Local\Temp/ipykernel_24724/3537972354.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_cases_lag['cases'  + str(i).zfill(2) + '_deriv2_norm'] = \


In [19]:
df_diff_state = df_diff.copy()
df_diff_state['fips_st'] = df_diff_state['fips'].str.slice(0,2) + '000'
df_diff_state = df_diff_state.groupby(['date', 'fips_st'], as_index = False).sum()

In [20]:
df_target_denom_state = pd.DataFrame()
for x in df_diff_state['date'].unique():
    df_diff_temp = df_diff_state.loc[df_diff_state['date'] <= x]
    
    sorted_val = df_diff_temp.loc[df_diff_temp['cases'] > 0].sort_values(by='date')
    recent_result_cases = sorted_val.drop_duplicates('fips_st', keep='last')

    df_diff_temp = df_diff_temp.loc[df_diff_temp['date'] == x]

    df_cases_target = df_diff_temp.merge(recent_result_cases[['date', 'fips_st', 'cases']], on = ['fips_st'], how = 'left')
    df_cases_target['days_since_gt0'] = df_cases_target['date_x'] - df_cases_target['date_y']
    df_cases_target.rename(columns={'date_x':'date', 'cases_y':'recent_cases_gt0'}, inplace=True)
    df_cases_target = df_cases_target.drop(columns = ['date_y', 'cases_x'])
    df_target_denom_state = df_target_denom_state.append(df_cases_target)

In [21]:
df_cases_lag_state = \
    df_diff_state.merge(df_target_denom_state[['date', 'fips_st', 'recent_cases_gt0', 'days_since_gt0']], on = ['date', 'fips_st'], how = 'inner')

In [22]:
df_left_state = df_cases_lag_state.copy()
for i in range(1, 31):
    df_left_state['date'] = df_left_state['date'] - pd.Timedelta(days = -1)
    df_cases_lag_state = df_cases_lag_state.merge(df_left_state[['date', 'fips_st', 'cases']], on = ['date', 'fips_st'], how = 'inner')
    df_cases_lag_state.rename(columns={'cases_y':'cases' + str(i).zfill(2)}, inplace=True)
    df_cases_lag_state.rename(columns={'cases_x':'cases'}, inplace=True)
df_cases_lag_state.rename(columns={'cases':'cases00'}, inplace=True)

In [23]:
for i in range(30):
    df_cases_lag_state['cases'  + str(i).zfill(2) + '_norm'] = df_cases_lag_state['cases' + str(i).zfill(2)] / df_cases_lag_state['recent_cases_gt0']
for i in range(29):
    df_cases_lag_state['cases'  + str(i).zfill(2) + '_deriv1_norm'] = \
        df_cases_lag_state['cases'  + str(i).zfill(2) + '_norm'] \
            - df_cases_lag_state['cases'  + str(i + 1).zfill(2) + '_norm']
for i in range(28):
    df_cases_lag_state['cases'  + str(i).zfill(2) + '_deriv2_norm'] = \
        df_cases_lag_state['cases'  + str(i).zfill(2) + '_deriv1_norm'] \
            - df_cases_lag_state['cases'  + str(i + 1).zfill(2) + '_deriv1_norm']

C:\Users\gordo\AppData\Local\Temp/ipykernel_24724/1049967788.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_cases_lag_state['cases'  + str(i).zfill(2) + '_deriv2_norm'] = \


In [24]:
df_diff_nation = df_diff.copy()
df_diff_nation = df_diff_nation.groupby(['date'], as_index = False).sum()

In [25]:
df_target_denom_nation = pd.DataFrame()
for x in df_diff_nation['date'].unique():
    df_diff_temp = df_diff_nation.loc[df_diff_nation['date'] <= x]
    
    sorted_val = df_diff_temp.loc[df_diff_temp['cases'] > 0].sort_values(by='date')
    recent_result_cases = sorted_val.tail(1)
    recent_result_cases = recent_result_cases.assign(days_since_gt0 = x - recent_result_cases['date'], date = x)
    
    recent_result_cases.rename(columns={'cases':'recent_cases_gt0'}, inplace=True)
    df_target_denom_nation = df_target_denom_nation.append(recent_result_cases)

In [26]:
df_cases_lag_nation = \
    df_diff_nation.merge(df_target_denom_nation[['date', 'recent_cases_gt0', 'days_since_gt0']], on = ['date'], how = 'inner')

In [27]:
df_left_nation = df_cases_lag_nation.copy()
for i in range(1, 31):
    df_left_nation['date'] = df_left_nation['date'] - pd.Timedelta(days = -1)
    df_cases_lag_nation = df_cases_lag_nation.merge(df_left_nation[['date', 'cases']], on = ['date', ], how = 'inner')
    df_cases_lag_nation.rename(columns={'cases_y':'cases' + str(i).zfill(2)}, inplace=True)
    df_cases_lag_nation.rename(columns={'cases_x':'cases'}, inplace=True)
df_cases_lag_nation.rename(columns={'cases':'cases00'}, inplace=True)

In [28]:
for i in range(30):
    df_cases_lag_nation['cases'  + str(i).zfill(2) + '_norm'] = df_cases_lag_nation['cases' + str(i).zfill(2)] / df_cases_lag_state['recent_cases_gt0']
for i in range(29):
    df_cases_lag_nation['cases'  + str(i).zfill(2) + '_deriv1_norm'] = \
        df_cases_lag_nation['cases'  + str(i).zfill(2) + '_norm'] \
            - df_cases_lag_nation['cases' + str(i + 1).zfill(2) + '_norm']
for i in range(28):
    df_cases_lag_nation['cases'  + str(i).zfill(2) + '_deriv2_norm'] = \
        df_cases_lag_nation['cases'  + str(i).zfill(2) + '_deriv1_norm'] \
            - df_cases_lag_nation['cases'  + str(i + 1).zfill(2) + '_deriv1_norm']

C:\Users\gordo\AppData\Local\Temp/ipykernel_24724/4077784464.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_cases_lag_nation['cases'  + str(i).zfill(2) + '_deriv2_norm'] = \


In [29]:
df_cases_lag.tail()

,date,county,state,fips,cases00,recent_cases_gt0,days_since_gt0,day_of_week,month,cases01,...,cases18_deriv2_norm,cases19_deriv2_norm,cases20_deriv2_norm,cases21_deriv2_norm,cases22_deriv2_norm,cases23_deriv2_norm,cases24_deriv2_norm,cases25_deriv2_norm,cases26_deriv2_norm,cases27_deriv2_norm
1278782,2021-06-01,Sweetwater,Wyoming,56037,47.0,47.0,0 days,1_Tues,6,0.0,...,-0.042553,0.021277,0.361702,-0.744681,0.446809,0.191489,-0.127660,-0.106383,-0.085106,0.468085
1278783,2021-06-01,Teton,Wyoming,56039,1.0,1.0,0 days,1_Tues,6,0.0,...,1.000000,1.000000,0.000000,-5.000000,4.000000,-2.000000,13.000000,-17.000000,5.000000,6.000000
1278784,2021-06-01,Uinta,Wyoming,56041,6.0,6.0,0 days,1_Tues,6,0.0,...,-0.333333,0.333333,1.333333,-3.500000,2.000000,1.000000,-1.666667,1.166667,0.166667,-0.166667
1278785,2021-06-01,Washakie,Wyoming,56043,3.0,3.0,0 days,1_Tues,6,0.0,...,-2.333333,1.000000,0.000000,-0.333333,0.333333,0.000000,0.000000,1.666667,-3.333333,2.666667
1278786,2021-06-01,Weston,Wyoming,56045,1.0,1.0,0 days,1_Tues,6,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [30]:
#Next Steps
#(0) Make 28 target variables, one for each day.
#(1) Make a 28-day forecast, one for 2021-06-01, one for 2021-07-01.
#(2) Predict completion, and then multiply by these values.
#(3) See if a surge exists for June in Springfield Missouri
#(4) See if a surge exists for July` for rest of United States

In [31]:
df_left = df_diff.copy()
df_left['date'] = df_left['date'] - pd.Timedelta(days = 1)
df_lookforward_01 = df_diff.merge(df_left[['date', 'fips', 'cases']], on = ['date', 'fips'], how = 'inner')
df_lookforward_01.rename(columns={'cases_y':'cases'}, inplace=True)
df_lookforward_01 = df_lookforward_01.drop(columns = ['cases_x'])

In [32]:
df_cases_completion = \
    df_target_denom.merge(df_lookforward_01[['date', 'fips', 'cases']], on = ['date', 'fips'], how = 'inner')
df_cases_completion['_target'] = df_cases_completion['cases'] / df_cases_completion['recent_cases_gt0']
df_cases_completion.loc[(df_cases_completion['_target'] < 0 ),'_target'] = 0
#Double log transform
df_cases_completion['_target'] = np.log1p(np.log1p(df_cases_completion['_target']))
df_cases_completion = \
    df_cases_completion.drop(columns = ['county', 'state', 'recent_cases_gt0', 'days_since_gt0', 'cases'])

In [34]:
df_long = pd.DataFrame()
for i in range(1,29):
    df_left = df_diff.copy()
    df_left['date'] = df_left['date'] - pd.Timedelta(days = i)
    df_lookforward = df_diff.merge(df_left[['date', 'fips', 'cases']], on = ['date', 'fips'], how = 'inner')
    df_lookforward.rename(columns={'cases_y':'cases'}, inplace=True)
    df_lookforward = df_lookforward.drop(columns = ['cases_x'])
    df_cases_completion = \
        df_target_denom.merge(df_lookforward[['date', 'fips', 'cases']], on = ['date', 'fips'], how = 'inner')
    df_cases_completion = df_cases_completion.assign(variable = '_target'+ str(i).zfill(2))
    df_cases_completion = \
        df_cases_completion.assign(val = df_cases_completion['cases'] / df_cases_completion['recent_cases_gt0'])
    df_cases_completion.loc[df_cases_completion['val'] < 0] = 0
    #Double log transform
    df_cases_completion['val'] = \
        np.log1p(np.log1p(df_cases_completion['val']))
    df_cases_completion = \
        df_cases_completion.drop(columns = ['county', 'state', 'recent_cases_gt0', 'days_since_gt0', 'cases'])
    df_long = df_long.append(df_cases_completion)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28


In [35]:
df_long.loc[(df_long['fips'] == '36999') & (df_long['variable'] == '_target14')].tail(20)

,date,fips,variable,val
1268872,2021-04-29 00:00:00,36999,_target14,0.387273
1272119,2021-04-30 00:00:00,36999,_target14,0.338234
1275365,2021-05-01 00:00:00,36999,_target14,0.188241
1278611,2021-05-02 00:00:00,36999,_target14,0.440449
1281857,2021-05-03 00:00:00,36999,_target14,0.341597
1285103,2021-05-04 00:00:00,36999,_target14,0.535735
1288349,2021-05-05 00:00:00,36999,_target14,0.267130
1291595,2021-05-06 00:00:00,36999,_target14,0.377619
1294841,2021-05-07 00:00:00,36999,_target14,0.400928
1298087,2021-05-08 00:00:00,36999,_target14,0.302321


In [36]:
#Test memory usage
import os, psutil
process = psutil.Process(os.getpid())
print(psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2)

4749.90234375


In [56]:
df_target_denom_nation.tail(5)

,date,recent_cases_gt0,days_since_gt0
783,2022-03-14,49636.0,0 days
784,2022-03-15,25738.0,0 days
785,2022-03-16,39384.0,0 days
786,2022-03-17,35842.0,0 days
787,2022-03-18,36267.0,0 days


In [104]:
df_cases_lag.loc[df_cases_lag['fips'] == '36999'].to_csv('test.csv')

In [12]:
#Code Template for Later

#keys_c = CategoricalDtype( sorted( full_df['unique_key'].unique()), ordered = True)
#var_c = CategoricalDtype( sorted( full_df.variable.unique()), ordered = True)
#row = full_df['unique_key'].astype(keys_c).cat.codes
#col = full_df['unique_key'].astype(var_c).cat.codes

#sparse_matrix = csr_matrix((full_df['val'], (row, col)), shape = keys_c.categories.size, var_c = categories.size)
#X = sparse_matrix[0:train_index, 1:]
#Y = pd.DataFrame(sparse_matrix[0:train_index, 0:].astype(np.int).todense())
#score_X = sparse_matrix[train_index:,1:]

NameError: name 'full_df' is not defined

In [12]:
#df_cases_completion.loc[df_cases_completion['fips'] == '48113'].tail()

,date,county,state,fips,recent_cases_gt0,days_since_gt0,cases,deaths,_target
2267295,2022-03-02,Dallas,Texas,48113,74.0,0 days,98.0,6.0,0.611628
2270547,2022-03-03,Dallas,Texas,48113,98.0,0 days,82.0,22.0,0.474985
2273801,2022-03-04,Dallas,Texas,48113,82.0,0 days,152.0,12.0,0.717158
2277055,2022-03-05,Dallas,Texas,48113,152.0,0 days,17.0,2.0,0.100766
2280308,2022-03-06,Dallas,Texas,48113,17.0,0 days,164.0,0.0,1.213512


In [64]:
#Testing for later
#for i in range(30):
#    df['date_shift' + str(i + 1).zfill(2)] = df['date'] + pd.Timedelta(days = i + 1)

,date,county,state,fips,cases,deaths,date_shift01,date_shift02,date_shift03,date_shift04,...,date_shift21,date_shift22,date_shift23,date_shift24,date_shift25,date_shift26,date_shift27,date_shift28,date_shift29,date_shift30


In [ ]:
#Need three columns
#A key (based on date and fips)
#A variable name
#A variable value
#Target variable will be percent increase